In [ ]:
!pip install jieba

In [2]:
# -*- coding: utf-8 -*-
import random
import numpy as np
import csv
import jieba


file_path = './data/review.csv'
jieba.load_userdict("./data/userdict.txt")


def load_corpus(corpus_path):
    with open(corpus_path, 'r') as f:
        reader = csv.reader(f)
        rows = [row for row in reader]

    # 将读取出来的语料转为list
    review_data = np.array(rows).tolist()
    # 打乱语料的顺序
    random.shuffle(review_data)

    review_list = []
    sentiment_list = []
    # 第一列为差评/好评， 第二列为评论
    for words in review_data:
        review_list.append(words[1])
        sentiment_list.append(words[0])

    return review_list, sentiment_list

review_list, sentiment_list = load_corpus(file_path)

print(review_list[:10], sentiment_list[:10])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.909 seconds.
Prefix dict has been built successfully.


['非常认同这个评论：傻逼右翼政治宣传电影如果中国人说他经典，我只能说你太装逼', '我是少數覺得很無聊的人', '虽然没有漫画好 还是看得很感动！>皿<。', '一气呵成地看完，太帅了！', '憋死了，憋死了，不好看！我最懒得想象一千年或两千年后世界怎么样·最懒得管你的真爱会不会变成刻在玄武岩上的楔形文字·总之你没有办法用旷日持久的时间打败我，如果你爱我，我只要一百年。不过这是题外。', '小清新，一步难一步佳，日子一样过。', '所以他值9.2的原因是？？我看不懂', '人生全靠政治正确', '其实感觉不是非常非常的感动，对于青年时期的描述有点浮于表面但是演得不错，对老年时期的刻画有点诡异，那句Your Mother is My Home 就显得造作了。时长放在那儿，总体不错。我在思考，要是当年是BS演了Alie，是不是歌坛就少了个女王陛下？！@Home', '青春的银幕会有不同的人出场，共同经历一些事，一起走过很多地方。伴随着成长，有些人离去了，即使短暂停留也让人终生难忘，有些人常在身边最初感觉已不再；有些事剪断了，只盼有机会拼凑重温；有些地方用以缅怀，终也难逃土崩瓦解的时代。青春散场后，那些本以为可以抛弃的却是最根深蒂固埋藏在心的。'] ['0', '0', '1', '1', '0', '1', '0', '0', '1', '0']


In [3]:
n = len(review_list) // 5

train_review_list, train_sentiment_list = review_list[n:], sentiment_list[n:]
test_review_list, test_sentiment_list = review_list[:n], sentiment_list[:n]

print('训练集数量： {}'.format(str(len(train_review_list))))
print('测试集数量： {}'.format(str(len(test_review_list))))

训练集数量： 41402
测试集数量： 10350


In [4]:
import re
import jieba


stopword_path = './data/stopwords.txt'


def load_stopwords(file_path):
    stop_words = []
    with open(file_path, encoding='UTF-8') as words:
       stop_words.extend([i.strip() for i in words.readlines()])
    return stop_words


def review_to_text(review):
    stop_words = load_stopwords(stopword_path)
    # 去除英文
    review = re.sub("[^\u4e00-\u9fa5^a-z^A-Z]", '', review)
    review = jieba.cut(review)
    # 去掉停用词
    if stop_words:
        all_stop_words = set(stop_words)
        words = [w for w in review if w not in all_stop_words]

    return words


# 用于训练的评论
review_train = [' '.join(review_to_text(review)) for review in train_review_list]
# 对于训练评论对应的好评/差评
sentiment_train = train_sentiment_list

# 用于测试的评论
review_test = [' '.join(review_to_text(review)) for review in test_review_list]
# 对于测试评论对应的好评/差评
sentiment_test = test_sentiment_list

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

count_vec = CountVectorizer(max_df=0.8, min_df=3)

tfidf_vec = TfidfVectorizer()

# 定义Pipeline对全部步骤的流式化封装和管理，可以很方便地使参数集在新数据集（比如测试集）上被重复使用。
def MNB_Classifier():
    return Pipeline([
        ('count_vec', CountVectorizer()),
        ('mnb', MultinomialNB())
    ])

mnbc_clf = MNB_Classifier()

In [6]:
mnbc_clf = MNB_Classifier()

# 进行训练
mnbc_clf.fit(review_train, sentiment_train)

# 测试集准确率
print('测试集准确率： {}'.format(mnbc_clf.score(review_test, sentiment_test)))

测试集准确率： 0.792463768115942
